# 朴素贝叶斯分类实现MNIST数据集手写数字识别

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

In [2]:
train_x, test_x, train_y, test_y = np.load("../_data/sklearnstyle_mnist_data.npz").values()
threshold = 0.5
train_x, test_x = (train_x > threshold).astype(np.int32), (test_x > threshold).astype(np.int32)
fobj = VarianceThreshold()
fobj.fit(train_x)
train_x = fobj.transform(train_x)
test_x = fobj.transform(test_x)
train_x.shape, test_x.shape

((60000, 676), (10000, 676))

In [3]:
dtrain = np.c_[train_x, train_y]
dtrain = pd.DataFrame(dtrain)
laplace = 0.001
dgrp = dtrain.groupby(dtrain.columns[-1])
fq   = (dgrp.size().to_numpy() + laplace) / (dtrain.shape[0] + 10 * laplace)
pvals = dgrp.agg(lambda x : (np.sum(x) + laplace) / (len(x) + 2 * laplace)).to_numpy()
npvals = 1 - pvals

In [4]:
train_y_pred = np.empty_like(train_y)
for i in range(train_x.shape[0]):
    train_y_pred[i] = np.argmax(fq * np.where(train_x[i] == 0, npvals, pvals).prod(axis=1))
from sklearn.metrics import accuracy_score
accuracy_score(train_y, train_y_pred)

0.8373833333333334

In [5]:
test_y_pred = np.empty_like(test_y)
for i in range(test_x.shape[0]):
    test_y_pred[i] = np.argmax(fq * np.where(test_x[i] == 0, npvals, pvals).prod(axis=1))
accuracy_score(test_y, test_y_pred)

0.8442